# Twitter Sentiment Hypothesis Generation

This notebook is adapted from `quickstart_local.ipynb` and contains the following modifications:

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5' # Set to the index of the GPU you want to use; see visible GPUs with `nvidia-smi` on command line

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from hypothesaes.quickstart import train_sae, interpret_sae, generate_hypotheses, evaluate_hypotheses
from hypothesaes.embedding import get_local_embeddings
from hypothesaes.llm_local import get_vllm_engine

current_dir = os.getcwd()
assert current_dir.endswith("phoenix")

## Dataset
The dataset I will be using is [Twitter Tweet Sentiments (27.5k)](https://www.kaggle.com/datasets/yasserh/twitter-tweets-sentiment-dataset), a collection of 27,480 tweets and the associated sentiments. 5496 of these tweets are used for validation during SAE training and 5496 tweets used for holdout evaluation. The target variable is the `sentiment` column, which can be `negative`, `neutral`, or `positive`, and we are interested in seeing what features of the `text` column predict it.

In [4]:
base_dir = os.path.join("data")
df = pd.read_csv(os.path.join(base_dir, "Tweets.csv"))
df = df[pd.notnull(df["text"])] # GPT generated command to clean :)

train_df, val_df = train_test_split(df, test_size=5496*2, train_size=16448, random_state=42)
val_df_SAE, val_df_holdout = train_test_split(val_df, test_size=5496, train_size=5496, random_state=42)

train_df.head(5)

,textID,text,selected_text,sentiment
19413,ba7bbe76fe,: saw it yesterday. Pretty good.,good.,positive
23147,09b5bef434,"hey, I can`t make it to Makers tonight","hey, I can`t make it to Makers tonight",neutral
21547,83cdebaa92,"Whats with you though, you sound a bit down y...",you sound a bit down yourself.,negative
14162,914da6164e,No B2G1 for me. Trying to save cash for next ...,No B2G1 for me. Trying to save cash for next ...,neutral
6474,d7709f9f53,hahahaha omg you win the internetz today! 'W...,a omg you win,positive


## 1. Feature Generation
First, we will compute the embeddings of the `text` column for the training and validation sets.

In [5]:
texts = train_df["text"].tolist()
sentiments = train_df["sentiment"].tolist()
val_texts = val_df["text"].tolist()

EMBEDDER = "nomic-ai/modernbert-embed-base"
CACHE_NAME = f"twitter_quickstart_local_{EMBEDDER}"

text2embedding = get_local_embeddings(texts + val_texts, model=EMBEDDER, batch_size=128, cache_name=CACHE_NAME)
train_embeddings = np.stack([text2embedding[text] for text in texts])
val_embeddings = np.stack([text2embedding[text] for text in val_texts])

Loaded model nomic-ai/modernbert-embed-base to cuda


Processing chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunk 0:   0%|          | 0/215 [00:00<?, ?it/s]

Saved 27440 embeddings to /home/phoenixw/HypotheSAEs/emb_cache/twitter_quickstart_local_nomic-ai/modernbert-embed-base/chunk_000.npy


Now that we have the embeddings, we will use a SAE wiil sparsify these representations. Since the size of the dataset used here and the dataset used in `quickstart_local.ipynb` are relatively similar, and after consulting the `README`, I decided to train a Matryoshka SAE with the same parameters $M=256$, $k=8$, and $\text{prefix\_lengths} = [32, 256]$.

In [6]:
checkpoint_dir = os.path.join(base_dir, "checkpoints", CACHE_NAME)
sae = train_sae(embeddings=train_embeddings, M=256, K=8, matryoshka_prefix_lengths=[32, 256], checkpoint_dir=checkpoint_dir, val_embeddings=val_embeddings)

  0%|          | 0/100 [00:00<?, ?it/s]

Early stopping triggered after 94 epochs
Saved model to data/checkpoints/twitter_quickstart_local_nomic-ai/modernbert-embed-base/SAE_matryoshka_M=256_K=8_prefixes=32-256.pt


In [ ]:
## 2. 